In [4]:
# import os
# os.chdir("birthrate_mtgp")
from jax import numpy as jnp
import numpy as np
import numpyro.distributions as dist
import jax.numpy as jnp
import numpyro
from numpyro.handlers import scope

In [7]:
import pandas as pd

# File paths
path_fertility = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data with 2024.csv"
path_2024 = "/Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/2024 Mother Age.csv"

# Step 1: Load both datasets
df_fert = pd.read_csv(path_fertility)
df_2024 = pd.read_csv(path_2024)

# Step 2: Standardize column names
df_2024.columns = df_2024.columns.str.strip().str.lower()
df_2024.rename(columns={'month': 'month', 'state': 'state', 'motherage': 'age', 'births': 'births'}, inplace=True)

# Step 3: Map age groups to match target columns
def map_age_group(age):
    if age in ['15-19', '20-24']:
        return 'births_age1524'
    elif age in ['25-29', '30-34']:
        return 'births_age2534'
    elif age in ['35-39', '40-44']:
        return 'births_age3544'
    else:
        return None

df_2024['age_group'] = df_2024['age'].apply(map_age_group)
df_2024 = df_2024.dropna(subset=['age_group'])

# Step 4: Compute bimonthly code (bmcode: 1 for Jan-Feb, ..., 6 for Nov-Dec)
df_2024['bmcode'] = ((df_2024['month'] - 1) // 2 + 1).astype(int)

# Step 5: Aggregate to state-bmcode-age_group level
df_agg = df_2024.groupby(['state', 'bmcode', 'age_group'])['births'].sum().unstack('age_group').reset_index()
df_agg['year'] = 2023

# Step 6: Reorder and match column order with original dataset
df_agg = df_agg[['state', 'year', 'bmcode', 'births_age1524', 'births_age2534', 'births_age3544']]

# Step 7: Replace 2023 rows in original data
df_fert_no2023 = df_fert[df_fert['year'] != 2023]
df_updated = pd.concat([df_fert_no2023, df_agg], ignore_index=True)

# Step 8: Save updated dataset
output_path = path_fertility.replace("fertility_data with 2024.csv", "fertility_data_updated.csv")
df_updated.to_csv(output_path, index=False)
print("✅ Updated data saved to:", output_path)


✅ Updated data saved to: /Users/shaokangyang/Library/CloudStorage/GoogleDrive-sky.ang510@gmail.com/My Drive/Code/dobbs_fertility/data/fertility_data_updated.csv
